## Database and Data Access with Pandas

In [1]:
import numpy as np
import pandas as pd
import sqlite3

We'll delete any old database we might have in this directory (e.g., left from the last time we han this notebood)

In [2]:
import os
try:
    os.remove("sdss.db")
except OSError:
    pass

Open the connection to our database:

In [3]:
con = sqlite3.connect("sdss.db")

Create the tables:

In [4]:
con.execute("""
CREATE TABLE sources (
	run	INTEGER,
	rerun	INTEGER,
	camcol	INTEGER,
	field	INTEGER,
	obj	INTEGER,
	type	INTEGER,

	ra	REAL,
	dec	REAL,
	psfMag_r	REAL,
	psfMag_g	REAL,
	psfMagErr_r	REAL,
	psfMagErr_g	REAL
);
""")

In [5]:
con.execute("""
CREATE TABLE runs (
	run	INTEGER,
	ra	REAL,
	dec	REAL,
	mjdstart	REAL,
	mjdend	REAL,
	node	REAL,
	inclination	REAL,
	mu0	REAL,
	nu0	REAL
);
""")

Load the runs.txt data set. Note how Pandas' read_csv has a multitude of options that allow us to load the file without having to edit it (or write Python code to parse it first)

In [6]:
runs = pd.read_csv('runs.txt', 
            sep=" ", header=None, skiprows=1, 
            names=['run', 'ra', 'dec', 'mjdstart', 'mjdend', 'node', 'inclination', 'mu0', 'nu0'],
            index_col = 'run')

In [7]:
sources = pd.read_csv('sample.csv', index_col=['run', 'rerun', 'camcol', 'field', 'obj'])

Now we can save this data set into a database:

In [8]:
#con.execute("drop table if exists runs")
runs.to_sql('runs', con, if_exists='append')

In [9]:
#con.execute("drop table if exists stars")
sources.to_sql('sources', con, if_exists='append')

In [10]:
con.close()
del runs
del sources

## Reading data from a database using Pandas

In [11]:
con = sqlite3.connect("sdss.db")

In [12]:
result = pd.read_sql("""
    SELECT
        sources.ra, sources.dec, sources.run, mjdstart
    FROM
        sources
    JOIN runs ON sources.run = runs.run
""", con)

In [13]:
result

,ra,dec,run,mjdstart
0,8.129444,26.626617,7757,54764.323971
1,8.127839,26.627246,7757,54764.323971
2,8.127323,26.625120,7757,54764.323971
3,24.516117,-1.165794,4288,52971.187293
4,24.517941,-1.179207,4288,52971.187293
5,24.518946,-1.159151,4288,52971.187293
6,36.471592,-1.060939,4136,52909.399761
7,36.471758,-1.137845,4136,52909.399761
8,36.471758,-1.137845,4136,52909.399761
9,24.683920,-1.236317,4288,52971.187293


## SQL-type operations with Pandas

With Pandas, you can do many of the manipulations you'd use SQL for, *assuming your entire dataset fits into memory*.

For example:

In [14]:
result.groupby("run").size().order()

/Users/mjuric/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


run
3835      3
1892      3
2267      3
2297      3
3959      3
1133      3
1484      3
3829      3
3697      3
3649      3
5385      3
6160      3
4191      3
6753      3
1891      3
3104      3
3354      3
4389      6
2142      6
5402      6
2347      6
2029      6
1927      6
1119      6
8049      6
4252      6
3636      6
6075      6
1888      6
1855      6
       ... 
4076    183
1755    186
3360    186
259     189
5814    195
7717    201
5398    204
5183    204
5410    216
4836    219
1458    222
4933    222
4576    225
4203    228
4822    231
4135    234
4134    246
756     249
3384    252
3900    252
6182    258
4207    261
6177    273
5194    285
4849    291
1345    315
6162    321
4874    342
4832    345
4828    381
dtype: int64

In [15]:
# The equivalent of:
#   SELECT run, AVG(ra), AVG(dec), AVG(mjdstart) FROM sources WHERE dec > 10 GROUP BY run

result[result.dec > 10].groupby("run").aggregate(np.mean)

,ra,dec,mjdstart
run,,,
1000,356.175132,58.079059,51458.204579
1011,126.951944,14.852107,51459.360744
1022,19.177197,14.450658,51463.300571
1035,22.595771,14.051204,51464.309251
1043,19.876180,14.407586,51465.293308
1119,135.083360,55.119289,51582.244170
1120,192.853627,64.462837,51582.267938
1133,117.725359,38.619556,51584.168596
1302,147.895732,53.238794,51633.218119


### Finding out more:

A nice SQL-vs-Pandas tutorial (video+notebooks) is at http://www.gregreda.com/2014/12/22/translating-sql-to-pandas/

See http://pandas.pydata.org/pandas-docs/stable/groupby.html for more details.